In [4]:
import numpy as np
import pandas as pd
import random
import datetime
import faker
import os

# Initialise Faker for UK-based data generation
fake = faker.Faker("en_GB")

# Utility function to generate random dates
def random_date(start, end):
    """Generate a random datetime between `start` and `end`."""
    return start + datetime.timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

# Setting number of iteration in each column
num_customers = 500
num_products = 300
num_suppliers = 50
num_orders = 1000
num_transactions = 1200

# Define directory to store CSV files
output_dir = "/Users/partht/Downloads"
os.makedirs(output_dir, exist_ok=True)

# Generate Customers Data (Nominal, Ordinal, Interval)
customers = []
for _ in range(num_customers):
    customers.append({
        "customer_id": _ + 1,
        "name": fake.name(),
        "email": fake.email(),
        "phone": fake.phone_number(),
        "postcode": fake.postcode(),
        "registration_date": fake.date_between(start_date="-5y", end_date="today"),
        "customer_type": random.choice(["Regular", "VIP", "New"]),
    })
customers_df = pd.DataFrame(customers)

# Generate Suppliers Data (Nominal)
suppliers = []
for _ in range(num_suppliers):
    suppliers.append({
        "supplier_id": _ + 1,
        "name": fake.company(),
        "phone": fake.phone_number(),
        "email": fake.email(),
        "city": fake.city(),
        "postcode": fake.postcode(),
    })
suppliers_df = pd.DataFrame(suppliers)

# Generate Products Data (Nominal, Ratio)
products = []
for _ in range(num_products):
    products.append({
        "product_id": _ + 1,
        "name": fake.word().capitalize() + " Product",
        "category": random.choice(["Electronics", "Furniture", "Clothing", "Books"]),
        "unit_price": round(random.uniform(5, 500), 2),
        "stock_quantity": random.randint(0, 100),
    })
products_df = pd.DataFrame(products)

# Generate Orders Data (Nominal, Ordinal, Interval)
orders = []
for _ in range(num_orders):
    order_date = fake.date_between(start_date="-2y", end_date="today")
    delivery_date = order_date + datetime.timedelta(days=random.randint(1, 10))
    orders.append({
        "order_id": _ + 1,
        "customer_id": random.randint(1, num_customers),
        "order_date": order_date,
        "delivery_date": delivery_date,
        "status": random.choice(["Pending", "Shipped", "Delivered", "Cancelled"]),
    })
orders_df = pd.DataFrame(orders)

# Generate Order Items Data (Nominal, Ratio) with Composite Key
order_items = []
unique_order_items = set()
for order in orders:
    num_items = random.randint(1, 5)  # Each order can have 1-5 items
    for _ in range(num_items):
        while True:
            product_id = random.randint(1, num_products)
            combination = (order["order_id"], product_id)
            if combination not in unique_order_items:
                unique_order_items.add(combination)
                order_items.append({
                    "order_id": order["order_id"],
                    "product_id": product_id,
                    "quantity": random.randint(1, 10),
                    "price": round(random.uniform(10, 300), 2),
                })
                break
order_items_df = pd.DataFrame(order_items)

# Generate Transactions Data (Nominal, Ordinal, Ratio)
transactions = []
for _ in range(num_transactions):
    transaction_date = fake.date_between(start_date="-2y", end_date="today")
    transactions.append({
        "transaction_id": _ + 1,
        "transaction_type": random.choice(["Credit", "Debit"]),
        "transaction_date": transaction_date,
        "amount": round(random.uniform(50, 5000), 2),
        "status": random.choice(["Completed", "Pending", "Failed"]),
    })
transactions_df = pd.DataFrame(transactions)

In [5]:
def add_duplicate_rows_with_new_ids(df, primary_key, duplicate_fraction=0.05):
    """
    Adds duplicate rows to the DataFrame but assigns new unique IDs to the primary key column.
    """
    # Calculate the number of duplicate rows to add
    num_duplicates = int(len(df) * duplicate_fraction)
    
    # Select random rows to duplicate (excluding the primary key column)
    duplicates = df.drop(columns=[primary_key]).sample(num_duplicates, random_state=42)
    
    # Generate new unique IDs for the duplicated rows
    max_id = df[primary_key].max()
    duplicates[primary_key] = range(max_id + 1, max_id + 1 + num_duplicates)
    
    # Append the duplicates to the original DataFrame
    return pd.concat([df, duplicates], ignore_index=True)


# Apply to datasets
customers_df = add_duplicate_rows_with_new_ids(customers_df, "customer_id", duplicate_fraction=0.05)
suppliers_df = add_duplicate_rows_with_new_ids(suppliers_df, "supplier_id", duplicate_fraction=0.05)
products_df = add_duplicate_rows_with_new_ids(products_df, "product_id", duplicate_fraction=0.05)
orders_df = add_duplicate_rows_with_new_ids(orders_df, "order_id", duplicate_fraction=0.05)

In [6]:
def remove_random_values_from_columns(df, exclude_columns, fraction=0.1):
    """
    Removes random individual values from columns in the DataFrame .

    """
    for col in df.columns:
        if col not in exclude_columns:
            # Number of values to nullify in the column
            num_values_to_nullify = int(len(df) * fraction)
            
            # Randomly select indices in this column to nullify
            indices_to_nullify = np.random.choice(df.index, size=num_values_to_nullify, replace=False)
            
            # Nullify the selected indices in the column
            df.loc[indices_to_nullify, col] = None
    return df


# Identify columns to exclude (containing 'id')
def get_columns_to_exclude(df):
    return [col for col in df.columns if 'id' in col.lower()]


# Apply the function to datasets
transactions_exclude_cols = get_columns_to_exclude(transactions_df)
suppliers_exclude_cols = get_columns_to_exclude(suppliers_df)
customers_exclude_cols = get_columns_to_exclude(customers_df)

transactions_df = remove_random_values_from_columns(transactions_df, exclude_columns=transactions_exclude_cols, fraction=0.1)
suppliers_df = remove_random_values_from_columns(suppliers_df, exclude_columns=suppliers_exclude_cols, fraction=0.1)
customers_df = remove_random_values_from_columns(customers_df, exclude_columns=customers_exclude_cols, fraction=0.1)

In [ ]:
# Save to CSV files in specified directory
customers_df.to_csv(os.path.join(output_dir, "customers.csv"), index=False)
suppliers_df.to_csv(os.path.join(output_dir, "suppliers.csv"), index=False)
products_df.to_csv(os.path.join(output_dir, "products.csv"), index=False)
orders_df.to_csv(os.path.join(output_dir, "orders.csv"), index=False)
order_items_df.to_csv(os.path.join(output_dir, "order_items.csv"), index=False)
transactions_df.to_csv(os.path.join(output_dir, "transactions.csv"), index=False)

print(f"Data generated and saved as CSV files in {output_dir}.")
